<a href="https://colab.research.google.com/github/ms-tii/Counterfeit_Medicine_CaseStudy/blob/main/Counterfeit_Medicine_CaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Counterfeit Medicines Sales Prediction


---



Data set urls from googledrive

training_data_rawURL = https://drive.google.com/file/d/1N_Chixa3BB9t4lACzCvOK3hMRlIG8UsY/view?usp=sharing

test_data_rawURL = https://drive.google.com/file/d/1yV1whAvD9SnahLXi7sCwaOtcBYkkag8p/view?usp=sharing

In [2]:
#Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc





In [3]:
#Loading dataset from CSV url

training_data_CSV_URL = 'https://drive.google.com/uc?export=download&id=1N_Chixa3BB9t4lACzCvOK3hMRlIG8UsY'
test_data__CSV_URL = 'https://drive.google.com/uc?export=download&id=1yV1whAvD9SnahLXi7sCwaOtcBYkkag8p'

training_data = pd.read_csv(training_data_CSV_URL)
test_data = pd.read_csv(test_data__CSV_URL)

display(training_data.head())
display(test_data.head())

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402


,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level
0,HLZ81,NaN,Area027,1983,85.5328,Antibiotics,mild,0.112747,CityLimits,Tier 3,Medium
1,ECE94,13.45,Area045,2000,257.1460,OralContraceptives,mild,0.144446,DownTown,Tier 2,Unknown
2,SAD14,7.10,Area045,2000,98.1172,Antipyretics,mild,0.144221,DownTown,Tier 2,Unknown
3,EQV63,18.30,Area010,1996,135.3730,Tranquilizers,mild,0.100388,MidTownResidential,Tier 3,Unknown
4,AIR10,NaN,Area019,1983,112.8016,OralContraceptives,mild,0.022585,MidTownResidential,Tier 1,Small


In [4]:
# Check for missing values
training_missing_values = training_data.isnull().sum()
display("Missing values in training dataset:", training_missing_values)
test_missing_values = test_data.isnull().sum()
display("Missing values in test dataset:", test_missing_values)

# Check for duplicate rows
training_duplicate_rows = training_data.duplicated().sum()
display("Number of duplicate rows in training dataset:", training_duplicate_rows)
test_duplicate_rows = test_data.duplicated().sum()
display("Number of duplicate rows in test dataset:", test_duplicate_rows)

'Missing values in training dataset:'

,0
Medicine_ID,0
Counterfeit_Weight,1166
DistArea_ID,0
Active_Since,0
Medicine_MRP,0
Medicine_Type,0
SidEffect_Level,0
Availability_rating,0
Area_Type,0
Area_City_Type,0


'Missing values in test dataset:'

,0
Medicine_ID,0
Counterfeit_Weight,297
DistArea_ID,0
Active_Since,0
Medicine_MRP,0
Medicine_Type,0
SidEffect_Level,0
Availability_rating,0
Area_Type,0
Area_City_Type,0


'Number of duplicate rows in training dataset:'

np.int64(0)

'Number of duplicate rows in test dataset:'

np.int64(0)

In [5]:
training_data["Medicine_ID"].nunique()

1557

In [6]:
training_data.shape

(6818, 12)

In [7]:
# ignoring two columns for improving results
for col in ("Medicine_ID", "Counterfeit_Weight"):
  training_data.drop(col, axis=1, inplace=True)
  test_data.drop(col, axis=1, inplace=True)

In [8]:
display(training_data.head())

,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402


In [1]:
for col in ("DistArea_ID", "Medicine_Type", "SidEffect_Level", "Area_Type", "Area_City_Type", "Area_dist_level"):
  temp = pd.get_dummies(training_data[col], prefix=col)
  training_data = pd.concat([training_data, temp], axis=1)
  training_data.drop([col], axis=1, inplace=True)

  temp = pd.get_dummies(test_data[col], prefix=col)
  test_data = pd.concat([test_data, temp], axis=1)
  test_data.drop([col], axis=1, inplace=True)

NameError: name 'pd' is not defined